In [5]:
#install dependencies and libraries
! pip install TTS
!sudo apt-get install espeak-ng
!pip install fastapi uvicorn pydantic pyngrok nest_asyncio
!pip install python-multipart
import subprocess
from fastapi import FastAPI,Response
from fastapi.middleware.cors import CORSMiddleware
from fastapi.responses import JSONResponse
from fastapi.responses import StreamingResponse,FileResponse
from pydantic import BaseModel
from IPython.display import Audio
import uvicorn
import shutil
import nest_asyncio
from pyngrok import ngrok
import base64
import tempfile

nest_asyncio.apply()

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
espeak-ng is already the newest version (1.50+dfsg-10).
0 upgraded, 0 newly installed, 0 to remove and 8 not upgraded.


In [3]:
# run <! tts --list_models> to see available models
model="tts_models/en/ljspeech/glow-tts" #download model of your choice

In [4]:
def tts_call(text: str, model: str=model):
  # Create a temporary file to save the output audio
  with tempfile.NamedTemporaryFile(suffix=".wav", delete=False) as temp_file:
    out_path = temp_file.name

  # Run the TTS command
  subprocess.run(["tts", "--model_name", model, "--text", text, "--out_path", out_path])

  # Return the path to the saved audio file
  return out_path

In [8]:
app = FastAPI()
app.add_middleware(
    CORSMiddleware,
    allow_origins=['*'],
    allow_credentials=True,
    allow_methods=['*'],
    allow_headers=['*'],
)

In [9]:
class TextRequest(BaseModel):
    text: str

class OutputText(BaseModel):
  answer: str

In [10]:
@app.get("/synthesize_audios")
def synthesize_audio(payload: TextRequest):
    # Call your TTS function to generate the audio file
    print("text:", payload.text)
    audio_file_path = tts_call(payload.text)

    # Return the audio file as the response
    return FileResponse(audio_file_path, media_type="audio/wav")

@app.get("/", response_model=OutputText)
async def root():
    return {"answer": "Hello from TTS"}

In [11]:
!ngrok config add-authtoken 2S28VLMNCGx4zukfhelsfj4jPHd_oErC1e1zpvJKZkpecfPZ #<place your access token here>

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [12]:
ngrok_tunnel = ngrok.connect(8000)
print("Public URL:", ngrok_tunnel.public_url)
uvicorn.run(app, port=8000)

INFO:     Started server process [7287]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


Public URL: https://3508-35-236-172-57.ngrok-free.app
text: Hello World
INFO:     154.160.18.62:0 - "GET /synthesize_audios HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [7287]
